In [1]:
import pandas as pd
import sys
sys.path.append('..')
from configure.settings import DBSelector

In [4]:
db = DBSelector().get_engine('db_reits','tencent-1c')

In [5]:
df = pd.read_sql('reits_history',con=db)

In [6]:
df.head()

,id,date,volume,open,high,low,close,chg,percent,turnoverrate,amount,code
0,1.0,2022-10-10,32944545,2.847,2.847,2.847,2.847,0.657,30.00,0.0,93793119.0,SZ180102
1,2.0,2022-10-11,46590865,2.902,2.902,2.656,2.665,-0.182,-6.39,0.0,127474999.0,SZ180102
2,3.0,2022-10-12,18922379,2.665,2.735,2.613,2.719,0.054,2.03,0.0,50817384.0,SZ180102
3,7.0,2022-10-13,8198337,2.704,2.729,2.680,2.710,-0.009,-0.33,0.0,22163622.0,SZ180102
4,8.0,2022-10-14,10037875,2.703,2.713,2.661,2.673,-0.037,-1.37,0.0,27018560.0,SZ180102


In [7]:
name_df = pd.read_sql('reits_name_mapper',con=db)

In [8]:
all_df = pd.merge(df,name_df,on='code',how='left')

In [9]:
all_df.head()

,id,date,volume,open,high,low,close,chg,percent,turnoverrate,amount,code,index,name
0,1.0,2022-10-10,32944545,2.847,2.847,2.847,2.847,0.657,30.00,0.0,93793119.0,SZ180102,19,华夏合肥高新REIT
1,2.0,2022-10-11,46590865,2.902,2.902,2.656,2.665,-0.182,-6.39,0.0,127474999.0,SZ180102,19,华夏合肥高新REIT
2,3.0,2022-10-12,18922379,2.665,2.735,2.613,2.719,0.054,2.03,0.0,50817384.0,SZ180102,19,华夏合肥高新REIT
3,7.0,2022-10-13,8198337,2.704,2.729,2.680,2.710,-0.009,-0.33,0.0,22163622.0,SZ180102,19,华夏合肥高新REIT
4,8.0,2022-10-14,10037875,2.703,2.713,2.661,2.673,-0.037,-1.37,0.0,27018560.0,SZ180102,19,华夏合肥高新REIT


In [31]:
result=[]
for code,sub_df in all_df.groupby('code'):
    sub_df['date']=pd.to_datetime(sub_df['date'],format='%Y-%m-%d')
    sub_df = sub_df.set_index('date',drop=True)
    # print(code,sub_df.index[0])
    tmp_dict = {}
    p = (sub_df['close'].iloc[-1] - sub_df['close'].iloc[0])/sub_df['close'].iloc[0]*100
    tmp_dict['code']=code
    tmp_dict['percent']=p
    tmp_dict['first_percent']=round(sub_df['percent'].iloc[0],2)
    tmp_dict['start']=sub_df.index[0]
    result.append(tmp_dict)

In [32]:
df = pd.DataFrame(result)

In [36]:
df=df.sort_values('percent',ascending=False)

In [37]:
total_df = pd.merge(df,name_df,on='code',how='left')

In [39]:
total_df.to_excel('reits.xlsx',encoding='utf8')